# NEM Open Data Extraction Tool

**The Objective**: to create a tool that can easily extract open data from nemweb.com.au and assemble it into an analysis-ready format.

* For the intitial testing phase, extracted data will be stored as CSV files
* Once the data extraction pipeline is set up, we can experiment with SQL databases

In [1]:
%load_ext autoreload
%autoreload 2

# Standard Python
import os
from io import BytesIO
from zipfile import ZipFile
import pandas as pd

# My modules
from nem_tracker import NEM_tracker
from nem_extract import NEM_extractor
import config
config.use('config.json')

Value for DATA_PATH has been set!


### Creating a custom JSON sitemap

We require a dynamic way of navigating the NEM website to locate and extract resources. A JSON file that tracks the data files in a given location on the NEM website could be helpful... 

* Define data directories i.e. a function to add new directories to JSON
  * Perhaps have a *comments* field that allows user to insert comments about the data resource
* Each data directory file has an additional flag whether the data has been extracted or not - maybe a CSV file might work better... (it could also convert the timestamps to datetime to allow datatime queries)

In [2]:
data_dir = os.getenv('DATA_PATH')
nem_get = NEM_extractor(data_dir)
nem_get.select_resource()
nem_get.selected_resource

'/Reports/Current/Operational_Demand/ACTUAL_DAILY/'

In [3]:
nem_get.load_tracker_df()
nem_get.current_tracker_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 60 entries, 0 to 59
Data columns (total 5 columns):
TIMESTAMP        60 non-null object
UPLOAD_DATE      60 non-null object
DOWNLOADED       60 non-null bool
DOWNLOAD_DATE    60 non-null object
URL              60 non-null object
dtypes: bool(1), object(4)
memory usage: 2.1+ KB


In [4]:
nem_get.time_range

['2020-02-05 00:00:00', '2020-04-04 00:00:00']

## Next steps:

* Need to use `DOWNLOADED` flag to extract data from URLs within `.time_range`
* Need to set `DOWNLOADED_DATE` to current timestamp after extraction
  * Also set `DOWNLOADED` flag to `True`
* Need to also set a directory for downloaded data
  * Ideally based off `file_name` for `.current_tracker_df`